<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/0703.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### align 整體形狀
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate

def compute_alignment_offsets(signals, reference_index=0):
    """對每段訊號與參考訊號做 cross-correlation，回傳對齊所需的 lag（offset）"""
    # cross-correlation 對齊的是整體形狀， 而不是特定值
    ref = signals[reference_index]
    offsets = []

    for sig in signals:
        corr = correlate(sig, ref, mode='full', method='fft') # 用fft加速  (method='direct'（時域）window 一格格滑動)
        # lag : 要把 sig 往右移多少（正值），才能和參考訊號對齊
        # (若 lag < 0：表示該訊號「比參考早出現)
        lag = np.argmax(corr) - (len(ref) - 1)
        offsets.append(lag)

    return offsets

def plot_aligned_signals(signals, offsets, reference_index=0):
    ref_offset = offsets[reference_index]

    plt.figure(figsize=(12, 5))
    for i, (sig, offset) in enumerate(zip(signals, offsets)):
        shift = offset - ref_offset  # 參考訊號的 shift = 0，其他依照差值
        ## 調整 對齊 所做的平移處理
        x = np.arange(len(sig)) - shift  # 這會調整 x 軸的起始點 (ex: sig1 往右移動了兩格(-(-2)))
        plt.plot(x, sig, label=f"Signal {i} (lag={offset})")

    plt.title("Correctly Aligned Signals (ref fixed)")
    plt.xlabel("Aligned Time Index")
    plt.grid(True)
    plt.legend()
    plt.show()

# 測試資料
signals = [
    np.array([0, 0, 1, 2, 3, 2, 1, 0]),  # ref
    np.array([1, 2, 3, 2, 1, 0]),     # lag = -2 (比 ref 早出現 2 個單位時間。)
    np.array([0, 1, 2, 3, 2, 1]),     # lag = -1 (比 ref 早出現 1 個單位時間。)
]

offsets = compute_alignment_offsets(signals)
plot_aligned_signals(signals, offsets, reference_index=0)


## 找 lag 方法

1. Normalized Cross-Correlation 👉 抵抗振幅差異
  - 若你的訊號振幅不同，但波形相似，可用z-score 正規化後再做互相關
  - 對不同能量的訊號更公平（如有雜訊）
  - 適合非同步振幅，但形狀趨勢相同的訊號
2. Peak Matching（峰值比對）👉 適合事件驅動訊號（如脈衝）
  - 找每個訊號的主要峰值（如最大值），根據它們的位置來對齊
  - 快速簡單， 適合訊號中有明顯尖峰、突變點

3. Phase Correlation（相位相關） 👉 頻域找平移量，常用於圖像/序列對齊
  - 這種方法是找頻域相位差，理論上對縮放、亮度變化更穩健，也可套用於一維訊號：
  - 頻域方法，不需時域滑動
  - 抵抗整體能量變化
  - 較適合長訊號、影像、週期性波形



In [ ]:
# 1. Normalized Cross-Correlation
from scipy.signal import correlate
from scipy.stats import zscore
sig_norm = zscore(sig)
ref_norm = zscore(ref)
corr = correlate(sig_norm, ref_norm, mode='full', method='fft')
lag = np.argmax(corr) - (len(ref) - 1)


# 2. Peak Matching（峰值比對）
peak_sig = np.argmax(sig)
peak_ref = np.argmax(ref)
lag = peak_sig - peak_ref

# 3.
def phase_correlation(sig, ref):
    # zero-pad to same length
    n = len(sig) + len(ref)
    f1 = np.fft.fft(sig, n)
    f2 = np.fft.fft(ref, n)
    cross_power = (f1 * np.conj(f2)) / np.abs(f1 * np.conj(f2))
    corr = np.fft.ifft(cross_power)
    lag = np.argmax(np.abs(corr))
    lag = lag if lag < n // 2 else lag - n
    return lag

| 方法                    | 適用狀況      | 是否會改變值 | 抵抗振幅差異 | 可加速    | 是否非線性對齊 |
| --------------------- | --------- | ------ | ------ | ------ | ------- |
| Cross-Correlation     | 一般對齊      | ❌      | 🚫     | ✅（FFT） | ❌       |
| Normalized Cross-Corr | 波形相似但強度不同 | ❌      | ✅      | ✅      | ❌       |
| Peak Matching         | 明顯脈衝      | ❌      | 🚫     | ✅      | ❌       |
| Phase Correlation     | 頻率一致的訊號   | ❌      | ✅      | ✅      | ❌       |
| DTW                   | 結構類似但速率不同 | ✅（非線性） | ✅      | ❌      | ✅       |
| Manual Anchoring      | 有明確特徵點    | ❌      | ✅      | ✅      | ❌       |


* 抵抗震幅差異 : 當兩條波形的整體強度（振幅 / 高度）不同，但形狀趨勢相似，這時還能準確找出它們的相似位置、對齊點。

### 一階微分

In [ ]:
x = np.array([10, 15, 20, 25, 30])
# 1. 向前分差  (要快、簡單對齊（如互相關）)
np.diff(x) # [5, 5, 5, 5]  # forward: x[n+1] - x[n]
# 2. 中心分差 Central difference (要精確模擬導數（如頻率/速度變化）)
(x[2:] - x[:-2]) / 2 # ➜ [5, 5, 5]